<a href="https://colab.research.google.com/github/Michila0/pose-detection/blob/main/VedioConvertImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Upload the Video to Google Colab

In [ ]:
from google.colab import files
uploaded = files.upload()

# Install Required Libraries

In [ ]:
!pip install opencv-python

# Extract Frames from the Video

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Load the video
video_path = list(uploaded.keys())[0]  # Get the uploaded video file name
cap = cv2.VideoCapture(video_path)

# Create a folder to save extracted images
image_save_folder = "extracted_images"
if not os.path.exists(image_save_folder):
    os.makedirs(image_save_folder)

# Extract frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save every nth frame
    if frame_count % 1 == 0:
        image_path = os.path.join(image_save_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"Saved {image_path}")

    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames.")

#  View Extracted Images

In [ ]:
import matplotlib.pyplot as plt
import os

# List all extracted images
image_files = os.listdir(image_save_folder)

# Display the first few images
for i, image_file in enumerate(image_files[:5]):  # Display first 5 images
    image_path = os.path.join(image_save_folder, image_file)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(image_file)
    plt.axis('off')
    plt.show()

# Pose Estimation + Motion Analysis
This method uses pose estimation to track the batsman's body joints and analyzes their motion to detect batting actions.

## Install Libaries

In [ ]:
!pip install mediapipe opencv-python numpy matplotlib

In [ ]:
!pip uninstall mediapipe -y # uninstall existing mediapipe
!pip install mediapipe==0.10.20 # install specific version

## Detect Batting Pose with MediaPipe
Use MediaPipe to track the batsman's body joints(e.g: arms, shoulders, wrists) and analyze their movement over time.


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import os

#Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Process frames from the extracted images folder
image_save_folder = "extracted_images"
image_files = sorted(os.listdir(image_save_folder))

# Store joint angles or positions
batting_frames = []

for image_file in image_files:
    image_path = os.path.join(image_save_folder, image_file)
    frame = cv2.imread(image_path)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect pose
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Extract key joints (e.g., right wrist, left wrist, shoulders)
        landmarks = results.pose_landmarks.landmark
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate bat swing angle (example logic)
        angle = np.arctan2(right_wrist[1] - left_wrist[1], right_wrist[0] - left_wrist[0])

        # Detect swing (threshold-based)
        if abs(angle) > 1.5:  # Adjust based on your video
            batting_frames.append(image_file)
            print(f"Batting action detected in {image_file}")

pose.close()

## Visualize Batting Action
Plot frames where batting is detected:

In [ ]:
for batting_frame in batting_frames[:3]:  # Show first 3 detected frames
    image_path = os.path.join(image_save_folder, batting_frame)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Batting Action")
    plt.axis('off')
    plt.show()